<a href="https://colab.research.google.com/github/Falhebshi/Data-Mining/blob/main/Phase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**[1] Problem**
The purpose of this project is to analyze and predict the productivity of garment factory employees using data mining techniques. The main goal is to classify whether employees can achieve their target productivity and to cluster employees based on similar productivity patterns. By applying classification and clustering techniques, we aim to analyze employees' data, predict their likelihood of meeting productivity targets, and understand different productivity patterns based on shared characteristics.


#**[2] Data Mining Task**

Classification:

For classification, the model will be trained to predict whether an employee meets their targeted productivity using attributes such as working hours, experience, overtime, incentives, and other relevant factors. The goal is to identify employees who will likely meet their productivity targets based on their characteristics.

Clustering:

For clustering, the model will group employees with similar productivity patterns. These clusters will be used to predict results for new employees based on the group they are most similar to, helping better manage employee performance.

#**[3] Data**
describe eda

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('garments_worker_productivity.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Raw_data.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#**[4] Data Preprocessing**
describe data preprocessing

#**[5] Data Mining Technique**
describe the classification and clustering techniques

#**[6] Evaluation Technique**

#**[7] Findings and Discussion**

#**[8] References**